#### Building CNN to Detect Facial Key Points

In [4]:
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms, models, datasets
from torchsummary import summary
import numpy as np, pandas as pd, os, glob, cv2
from torch.utils.data import TensorDataset,DataLoader,Dataset
from copy import deepcopy
from mpl_toolkits.mplot3d import Axes3D

In [47]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import cluster
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from sklearn.model_selection import train_test_split

In [6]:
#!git clone https://github.com/udacity/P1_Facial_Keypoints.git

In [49]:
data = pd.read_csv('data/data/training_frames_keypoints.csv')

In [52]:
class FaceData(Dataset):
    def __init__(self,df):
        super(FaceData).__init__()
        self.df = df
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    def __len__(self):
        return len(self.df)
    def __getitem__(self,indx):
        img_path = 'data/data/training/' + self.df.iloc[indx,0]
        img = cv2.imread(img_path)/255
        keypoints = deepcopy(self.pd.iloc[indx,1:].tolist())
        keypoints_x = (np.array(keypoints[0::2])/img.shape[1]).tolist()
        keypoints_y = (np.array(keypoints[0::2])/img.shape[1]).tolist()
        keypoints_2 = keypoints_x+keypoints_y
        keypoints_2 = torch.tensor(keypoints_2)
        img = self.preprocess_input(img)
        
        return img, keypoints_2
    def preprocess_input(self,img):
        img = cv2.resize(img, (224,224))
        img = torch.tensor(img).permute(2,0,1)
        img = self.normalize(img).float()
        
        return img.to(device)
    def load_image(self,indx):
        img_path = 'data/data/training/' + self.df.iloc[indx,0]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
        img = cv2.resize(img, (224,224))
        
        return img
        

In [53]:
train_data, test_data  = train_test_split(data,test_size=0.2,random_state=101)

In [55]:
train_dataset = FaceData(train_data.reset_index(drop=True))
test_dataset = FaceData(test_data.reset_index(drop=True))

In [56]:
train_loader = DataLoader(train_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [57]:
def build_model():
    model = models.vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.avgpool = nn.Sequential( nn.Conv2d(512,512,3),
                                  nn.MaxPool2d(2),
                                  nn.Flatten())
    model.classifier = nn.Sequential(
        nn.Linear(2048, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 136),
        nn.Sigmoid())
    loss_function = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    return model.to(device), loss_function, optimizer

In [58]:
model, loss_function,optimizer = build_model()

/home/m294/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/m294/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
